In [2]:
%matplotlib inline
import requests as http
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import lib.lastfm as lastfm
import lib.curator as curator
#from lib.spotify import User
import time
import calendar
import datetime
import re
import unicodedata
import importlib
import requests as http
import json
import uuid
import redis as rd
import tornado
import pickle

In [2]:
redis = rd.StrictRedis(host='localhost', port=6379, db=0)
user_binary = redis.get('bornofawesomeness-obj')
user = pickle.loads(user_binary)

In [3]:
count = 0
artists = user.artists
cap = len(artists)
last_key = 'b333a19b2c0397e8e4c1224b49b3e7cd'
last_genre_method = 'artist.gettopTags'
last_base = 'http://ws.audioscrobbler.com/2.0/?method={0}&api_key={1}&format=json'
last_genre_url = last_base.format(last_genre_method, last_key) + '&autocorrect=1&artist='
def normalize_tags(tags, index):
    try:
        string = tags[index]['name']
        result = re.findall(r'([A-z])', string)
        return ''.join(result).lower()
    except IndexError:
        return None
genres = {}
start_time = time.clock()
for artist in artists:
    count += 1
    url = last_genre_url + artist
    url = url.replace(' ', '+')
    response = http.get(url, timeout=200).json()
    try:
        tags = response['toptags']['tag']
        genres[artist] = [
            normalize_tags(tags, 0),
            normalize_tags(tags, 1)
        ]
    except KeyError:
        genres[artist] = [None, None]

time.clock() - start_time, 'seconds'

(4.2668, 'seconds')

In [9]:
from tornado import ioloop
from tornado.httpclient import AsyncHTTPClient, HTTPRequest
genres = {}
left_to_process = len(user.artists)
def handle_response(response):
    if response.error:
        print("Error:", response.error, response.effective_url)
    else:
        try:
            res_json = json.loads(response.body.decode('utf-8'))
            tags = res_json['toptags']['tag']
            genres[artist] = [
                normalize_tags(tags, 0),
                normalize_tags(tags, 1)
            ]
        except KeyError:
            genres[artist] = [None, None]
    global left_to_process
    left_to_process -= 1
    if left_to_process == 0:
        print(time.clock() - start_time, 'seconds')

http_client = AsyncHTTPClient()
start_time = time.clock()
for artist in user.artists:
    url = last_genre_url + artist
    url = url.replace(' ', '+')
    req = HTTPRequest(url, request_timeout=200)
    http_client.fetch(req, handle_response)

1.836879999999999 seconds


In [11]:
arr = [np.nan] * 100
a = pd.Series(arr)
np.isnan(a[0])

True

In [16]:
class BadFilter(Exception):
    pass
        
def foo():
    raise BadFilter()
    
try:
    foo()
except BadFilter:
    print('woops')

woops
